In [ ]:
import re 

def generate_unique_placeholder(index):
    # Generate a unique placeholder based on the index
    return f"${index}$"

def split_input(input: str):
    # Step 1: Find all elements inside square brackets
    matches = re.findall(r'\[.*?\]', input)
    match_dict = {}
    for index, match in enumerate(matches):
        placeholder = generate_unique_placeholder(index)
        match_dict[placeholder] = match
        input = input.replace(match, placeholder)
    
    # Step 2: Split the rest of the string into individual characters
    split_list = list(input)
    
    # Step 3: Replace placeholders with original elements inside brackets
    for i, char in enumerate(split_list):
        if char in match_dict:
            split_list[i] = match_dict[char]
    
    return split_list

'''
This preporcessing is applied to the regex before applying shunting yard algorithm in order to : 
1) reduce all the regex to these operations only ( * , | , parenthes , concat )
2) add a concatination symbol between characters to be recognized by the algorithm as an operator 
3) replace every [match] with one char to be treated as other alphanumeric 
'''  
#^JOJO : Ana bas hatet epsilon lma kona bn sub a? with (a|) 3shan n3raf el wala haga wehna bn implement ~ 
#^JOJO : Shelt bas el while el taht w 5leto klo regex bas momkn n test brdo tany ehtyaty 
#^JOJO : Zbat hwar el matches da 
#TODO : store $  in the dict and replace with the value
#TODO : Add $ to the alpha numeric in the forloop
#TODO : Handle the dot operator 
def preprocessing(input : str) :
# input= 'a?a((cd)|(a|b))b+bb' 
    #^Step1: Replace zero or one symbol '?'
    step_1 = re.sub(r'(\w)\?', r'(\1|~)', input)
    #^Step2: Replace one or more symbol '+'
    step_2 = re.sub(r'(\w)\+', r'\1\1*', step_1)
    #^Step3: Add concat symbol before every [ or (  if they are not at the start of the regex and they are not preceded by ?
    pattern_before = re.compile(r'''
        (?<!^)      # Negative lookbehind assertion to ensure the position is not at the start of the string
        (?<!\?)     # Negative lookbehind assertion to ensure the position is not preceded by '?'
        (?=[\[\(])  # Positive lookahead assertion to match '[' or '(' without consuming them
    ''', re.VERBOSE)
    step_3 = pattern_before.sub('?', step_2) 
    #^Step 4: Add concat symbol after every ] or ) if they are not the end of regex OR they are not followed by * and not followed by ? 
    pattern_after = re.compile(r'''
        (?<=[\]\)])  # Positive lookbehind assertion to match ']' or ')' without consuming them
        (?!$)        # Negative lookahead assertion to ensure the position is not at the end of the string
        (?![\*\?])   # Negative lookahead assertion to ensure the position is not followed by '*' or '?'
    ''', re.VERBOSE)
    step_4 = pattern_after.sub('?', step_3)
    #^Step 5 : Add concat after every * if its not the end of regex and its not follwed by star 
    pattern_star = re.compile(r'''
        \*          # Match the '*' character
        (?!$)       # Negative lookahead assertion to ensure the position is not at the end of the string
        (?![\?])    # Negative lookahead assertion to ensure the position is not followed by '?'                      
    ''', re.VERBOSE)
    step_5 = pattern_star.sub('*?', step_4)
    #^Step 6 : Add concat after every alphanum or dot if its followed by alphanumeric or dot 
    pattern_alnum_dot = re.compile(r'''
        ([a-zA-Z0-9\.])  # Match any alphanumeric character or dot
        (?=[a-zA-Z0-9\.])  # Positive lookahead assertion to ensure it is followed by another alphanumeric character or dot
    ''', re.VERBOSE)
    step_6 = pattern_alnum_dot.sub(r'\1?', step_5)
    # removed_match_brackets = removed_match_brackets + '^'
    # i = 0 
    # while (1):
    #     if ( removed_match_brackets [i] == '^' ) : break
    #     if  ( removed_match_brackets[i+1].isalnum()  or   removed_match_brackets[i+1] == '(')  and ( removed_match_brackets[i].isalnum() or removed_match_brackets[i] == ')' or removed_match_brackets[i] == '*' ) :
    #         removed_match_brackets = removed_match_brackets[:i+1] + '?' + removed_match_brackets[i+1:]
    #     i = i + 1
    # print (removed_match_brackets)
    print (step_6)
    return split_input(step_6) 
print (preprocessing('aa[0-9]b'))


a?a?[0-9]?b
['a', '?', 'a', '?', '$', '0', '$', '?', 'b']


In [41]:
# Reeference : https://medium.com/@gregorycernera/converting-regular-expressions-to-postfix-notation-with-the-shunting-yard-algorithm-63d22ea1cf88
'''
Postfix notation removes the need for parentheses and allows computer programs to read in 
mathematical expressions one symbol after the other, instead of worrying about operator precedence 
and parentheses during computation. 

'''
def shuntingYard(input:str) :
    precedence_dict = {'*': 3, '?': 2, '|': 1}
    out =[]
    operator_stack = []
    for char in input :
        # If the input is alphanumeric then append to the output regex 
        if char.isalnum() :
            out.append(char)
        # If the input is an operator
        elif  char in precedence_dict.keys() :
            # The first operator in the stack 
            if len(operator_stack) == 0:
                operator_stack.append(char) 
            #  Any operator shouldn't be compared to an opening parenthes , if an opening parenthes is on the top of the stack Just add the char to the stack directly 
            elif operator_stack[-1] =='('or precedence_dict[ operator_stack[-1] ] < precedence_dict[char] :
                operator_stack.append(char)
            # If the operator on the top of the stack is the same as the current char then pop one to the output regex and leave the other in tha stack 
            # If two consecutive opening parenthes comes we need them both to be in the stack and not popped to the output because they will be deleted later 
            elif operator_stack[-1] != '(' and precedence_dict[ operator_stack[-1] ] == precedence_dict[char] : 
                out.append(char) 
            #If the operator at the top of the stack has higher precedence that the current operator then pop to the output until we can push the current operator to the stack 
            else : 
                while (len(operator_stack)>0 and operator_stack[-1] != '('and precedence_dict[ operator_stack[-1] ]  >= precedence_dict[char]) :
                    popped_operator = operator_stack.pop()
                    out.append(popped_operator)
                operator_stack.append(char) 
        elif char == '(' :
            operator_stack.append(char)
        # The current char is closing parenthes -> pop from the operator stack to the output until you reach an opening parenthes
        elif char == ')' :
            while operator_stack:
                operator = operator_stack.pop()
                if operator == '(':
                    break
                out.append(operator)
        # print ("parsing char :",char,"output:",out,"stack",operator_stack)
        # print("************************")
    # out.extend(operator_stack[::-1]) 
    # print ("Final out :",out)
    return out
    # print("Final stack :",operator_stack)


In [ ]:
import json

class Transition:
    def __init__(self, destination, input):
        self.destination = destination
        self.input = input

class State:
    state_counter = 0  

    def __init__(self):
        self.state_name = 'S'+ str(State.state_counter) 
        State.state_counter += 1
        self.transitions = [] 
        self.isTerminatingState = False 

    def add_transition(self, destination, input):
        self.transitions.append(Transition(destination, input))

    def to_dict(self):
        state_dict = {
            "isTerminatingState": self.isTerminatingState
        }
        for transition in self.transitions:
            state_dict[transition.input] = transition.destination.state_name
        return state_dict 
    
class NFA:
    def __init__(self):
        self.states = []
        self.start_state = None
        self.accept_states = []

    def add_state(self, state , is_start = False , is_accept = False  ):
        self.states.append(state)
        if is_start:
            self.start_state = state
        if is_accept:
            self.accept_states.append(state)
        return state
    
    def to_dict(self):
        nfa_dict = {
            "startingState": self.start_state.state_name if self.start_state else None
        }
        for state in self.states:
            nfa_dict[state.state_name] = state.to_dict()
        return nfa_dict

    def __repr__(self):
        return json.dumps(self.to_dict(), indent=2)

In [45]:
def alphanumeric_nfa(char) : 
    start_state = State() 
    end_state = State() 
    end_state.isTerminatingState = True
    start_state.add_transition(end_state , char)
    nfa = NFA() 
    nfa.add_state(start_state , is_start = True )
    nfa.add_state(end_state , is_start = False , is_accept = True) 
    return nfa

def constructNFA(input : str) : 
    stack_NFA = []
    for char in input : 
        if char.isalnum() : 
            stack_NFA.append(alphanumeric_nfa(char)) 
    return stack_NFA 
def ThomsonsConstruction (input : str) : 
    return constructNFA(shuntingYard(preprocessing(input)))
nfa_result = ThomsonsConstruction('abc')
for nfa in nfa_result:
    print(nfa) 

abc
a?b?c^
{
  "startingState": "S8",
  "S8": {
    "isTerminatingState": false,
    "a": "S9"
  },
  "S9": {
    "isTerminatingState": true
  }
}
{
  "startingState": "S10",
  "S10": {
    "isTerminatingState": false,
    "b": "S11"
  },
  "S11": {
    "isTerminatingState": true
  }
}
{
  "startingState": "S12",
  "S12": {
    "isTerminatingState": false,
    "c": "S13"
  },
  "S13": {
    "isTerminatingState": true
  }
}
